<a href="https://colab.research.google.com/github/v4nema/bert_intent_recognition/blob/main/BERT_from_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! rm -rf /content/bert_intent_recognition

In [ ]:
! git clone https://github.com/v4nema/bert_intent_recognition.git

import sys
sys.path.insert(1, "/content/bert_intent_recognition")

In [ ]:
! pip install transformers
! pip install evaluate
! pip install datasets

In [ ]:
import json
import os
import time
from typing import List

from transformers import AutoTokenizer, pipeline, Pipeline
from datasets import load_dataset, DatasetDict, Dataset
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
import evaluate

import numpy as np

In [ ]:
from train_intent_recognition import *
from test_intent_recognition import *
import config

config.dataset_files = ['bert_intent_recognition/dataset/part1.tsv', 'bert_intent_recognition/dataset/part2.tsv']
config.train_test_split_path = 'bert_intent_recognition/dataset'
config.results_path = 'bert_intent_recognition/results'
config.num_epochs = 50

In [ ]:
dataset_files = ['bert_intent_recognition/dataset/part1.tsv',
                 'bert_intent_recognition/dataset/part2.tsv']

dataset_dict = load_split_dataset(dataset_files,
                                  'bert_intent_recognition/dataset' if config.save_train_test_split
                                  else None)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Casting to class labels:   0%|          | 0/3098 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
id2label, label2id = map_id_label(dataset_dict)
len_dataset_dict_train = len(dataset_dict["train"])
model = create_model(id2label, label2id, len_dataset_dict_train)
pipe = train(dataset_dict, model, 'ckpt/best_model')

In [ ]:
# def test(dataset: Dataset, pipe: Pipeline, results_path: str):
#     task_evaluator = evaluate.evaluator("text-classification")
#     print('Computing evaluation results... It takes a lot, be patient...')
#     result = task_evaluator.compute(
#         model_or_pipeline=pipe,
#         data=dataset,
#         label_mapping=pipe.model.config.label2id
#     )
#     print('Evaluation completed.')
#     file = os.path.join(results_path, f'{str(int(time.time()))}_evaluation.json')
#     with open(file, "w") as f:
#         json.dump(result, f)
#     return result

In [ ]:
pipe.save_pretrained('bert_intent_recognition/pipe')
results_path = 'bert_intent_recognition/results'

In [ ]:
def test(dataset: Dataset, pipe: Pipeline, results_path: str):
    task_evaluator = evaluator("text-classification")
    print('Computing evaluation results... It takes a lot, be patient...')
    result = task_evaluator.compute(
        model_or_pipeline=pipe,
        data=dataset,
        label_mapping=pipe.model.config.label2id
    )
    print('Evaluation completed.')
    file = os.path.join(results_path, f'{str(int(time.time()))}_evaluation.json')
    with open(file, "w") as f:
        json.dump(result, f)

    return result


def save_scores(dataset: Dataset, pipe: Pipeline, results_path: str):
    res = []
    for text in dataset['text']:
        p_res = pipe(text, top_k=101)
        obj = {
            'text': text,
            'results': p_res
        }
        # obj.update({'results': p_res})
        res.append(obj)

    file = os.path.join(results_path, f'{str(int(time.time()))}_score.json')
    with open(file, "w") as f:
        json.dump(res, f)


In [ ]:
ds_test = Dataset.from_csv(f'{config.train_test_split_path}/test.csv', sep='\t')
result = test(ds_test, pipe, results_path)
save_scores(ds_test, pipe, results_path)
print(result)

Computing evaluation results... It takes a lot, be patient...
Evaluation completed.
{'accuracy': 0.9274193548387096, 'total_time_in_seconds': 260.14336635200016, 'samples_per_second': 2.3833012107680562, 'latency_in_seconds': 0.41958607476129056}


In [ ]:
! zip -r /content/drive/MyDrive/bert_intent_recognition.zip bert_intent_recognition

In [ ]:
text = "mi spieghi i titoli azionari" #@param {type:"string"}

31 37_share_education_share


3.1205695

In [ ]:
res = []
for text in ds_test['text']:
  p_res = pipe(text, top_k=101)
  obj = {
      'text': text,
      'results': p_res
  }
  # obj.update({'results': p_res})
  res.append(obj)

# score = pipe(ds_test['text'], top_k=101)

In [ ]:
file = os.path.join(results_path, f'{str(int(time.time()))}_score.json')
with open(file, "w") as f:
    json.dump(res, f)